In [1]:
import pickle
import pandas as pd

from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [2]:
with open("recommender_predictions.pkl", "rb") as f3:
    recs = pickle.load(f3)
with open('user_product_long.pkl', 'rb') as f:
    df = pickle.load(f)
with open('recommender_algorithm.pkl', 'rb') as f:
    algo = pickle.load(f)
with open('book_titles_map.pkl', 'rb') as f:
    book_titles = pickle.load(f)

In [29]:
with open('book_topics.pkl', 'rb') as f:
    topics = pickle.load(f)

In [3]:
df.head()

,bookid,userid,rating,title
38635884,220340,51312,3.0,Praisesong for the Widow
3108305,1103,51312,4.0,Snow Flower and the Secret Fan
58408324,33917,51312,4.0,The Namesake
68638650,5439,51312,3.0,Interpreter of Maladies
4306809,1138475,52807,3.0,Constantine


In [4]:
df.dtypes

bookid     object
userid      int64
rating    float64
title      object
dtype: object

In [5]:
algo

In [6]:
recs

[Prediction(uid=251417, iid='8694', r_ui=5.0, est=4.298857382466175, details={'was_impossible': False}),
 Prediction(uid=62161919, iid='25300956', r_ui=5.0, est=5, details={'was_impossible': False}),
 Prediction(uid=2876676, iid='30246', r_ui=5.0, est=4.102731446867163, details={'was_impossible': False}),
 Prediction(uid=8390907, iid='40217', r_ui=4.0, est=3.7838639417528115, details={'was_impossible': False}),
 Prediction(uid=1733488, iid='68429', r_ui=4.0, est=3.8076680204073243, details={'was_impossible': False}),
 Prediction(uid=310539, iid='9067850', r_ui=5.0, est=3.6494255102773954, details={'was_impossible': False}),
 Prediction(uid=1454825, iid='2', r_ui=3.0, est=4.3873981632351144, details={'was_impossible': False}),
 Prediction(uid=2592041, iid='46756', r_ui=3.0, est=3.96962233342499, details={'was_impossible': False}),
 Prediction(uid=3141031, iid='6185', r_ui=4.0, est=3.5900122206356815, details={'was_impossible': False}),
 Prediction(uid=22448098, iid='12013', r_ui=5.0, es

In [30]:
topics.head()

,bookid,topicid
0,39791289,0
1,23300,0
2,12446825,0
3,16091011,0
4,393147,0


In [31]:
userid = 10015197
books_read = df[['rating','title','bookid']][df['userid']==userid].drop_duplicates()
books_read.head()

,rating,title,bookid
43903676,5.0,"Holiday in Death (In Death, #7)",238125
54764863,5.0,"Blue Moon (Nightcreature, #1)",304295
44419835,5.0,"Dragon Wytch (Otherworld/Sisters of the Moon, #4)",2420112
50925077,5.0,"Shadow Kiss (Vampire Academy, #3)",2802316
77964708,5.0,"Jurassic Park (Jurassic Park, #1)",7677


In [36]:
books_read['bookid'] = [int(X) for X in books_read['bookid']]

In [37]:
pd.merge(books_read, topics, on = 'bookid', how='left')

,rating,title,bookid,topicid
0,5.0,"Holiday in Death (In Death, #7)",238125,0.0
1,5.0,"Blue Moon (Nightcreature, #1)",304295,0.0
2,5.0,"Dragon Wytch (Otherworld/Sisters of the Moon, #4)",2420112,0.0
3,5.0,"Shadow Kiss (Vampire Academy, #3)",2802316,0.0
4,5.0,"Jurassic Park (Jurassic Park, #1)",7677,NaN
5,5.0,"Night Myst (Indigo Court, #1)",7046413,0.0
6,5.0,"Twilight (Twilight, #1)",41865,0.0
7,5.0,Harvest Hunting (Otherworld/Sisters of the Moo...,7607561,0.0
8,5.0,Darkling (Otherworld/Sisters of the Moon #3),1704159,0.0
9,5.0,"Naked in Death (In Death, #1)",268602,0.0


In [8]:
items = []
for index, row in books_read.iterrows():
    row_dict = {}
    row_dict['rating'] = (u'\u2606'*int(row['rating']))
    row_dict['title'] = row['title']
    items.append(row_dict)
items

[{'rating': '☆☆☆☆☆', 'title': 'Holiday in Death (In Death, #7)'},
 {'rating': '☆☆☆☆☆', 'title': 'Blue Moon (Nightcreature, #1)'},
 {'rating': '☆☆☆☆☆',
  'title': 'Dragon Wytch (Otherworld/Sisters of the Moon, #4)'},
 {'rating': '☆☆☆☆☆', 'title': 'Shadow Kiss (Vampire Academy, #3)'},
 {'rating': '☆☆☆☆☆', 'title': 'Jurassic Park (Jurassic Park, #1)'},
 {'rating': '☆☆☆☆☆', 'title': 'Night Myst (Indigo Court, #1)'},
 {'rating': '☆☆☆☆☆', 'title': 'Twilight (Twilight, #1)'},
 {'rating': '☆☆☆☆☆',
  'title': 'Harvest Hunting (Otherworld/Sisters of the Moon #8)'},
 {'rating': '☆☆☆☆☆', 'title': 'Darkling (Otherworld/Sisters of the Moon #3)'},
 {'rating': '☆☆☆☆☆', 'title': 'Naked in Death (In Death, #1)'},
 {'rating': '☆☆☆☆☆', 'title': 'Glory in Death (In Death, #2)'},
 {'rating': '☆☆☆☆☆', 'title': 'Angels & Demons  (Robert Langdon, #1)'},
 {'rating': '☆☆☆☆☆',
  'title': 'Blood Wyne (Otherworld/Sisters of the Moon, #9)'},
 {'rating': '☆☆☆☆☆', 'title': 'Morning Glory'},
 {'rating': '☆☆☆☆☆', 'title

In [9]:
fantasy_books = list(book_titles.keys())
fantasy_books[0:5]

['39791289', '23300', '12446825', '16091011', '393147']

In [10]:
books_not_read = list(set(fantasy_books) - set(list(df['bookid'][df['userid']==userid])))
books_not_read[0:5]
len(books_not_read)

45960

In [11]:
estimates = []
for bookid in books_not_read:
    uid = userid
    iid = bookid   
    estimates.append(algo.predict(uid, iid, verbose=False))

In [12]:
book_recs = pd.DataFrame(estimates).sort_values(by='est', ascending = False)
book_recs['title'] = book_recs['iid'].map(book_titles)
book_recs = book_recs[(book_recs['est'] == 5.0)]
for i in range(2,20):
    book_recs = book_recs[book_recs['title'].str.contains('#'+str(i))==False]

In [13]:
book_rating_aggs = pd.read_pickle('fantasy_books_info_dataframe.pkl')
book_rating_aggs.head()

,average_rating,bookid,ratings_count,title
0,4.48,39791289,21.0,Upon Broken Wings
1,3.92,23300,2893.0,"The Door to Time (Ulysses Moore, #1)"
2,4.38,12446825,13511.0,"Feversong (Fever, #9)"
3,4.50,16091011,5.0,The Sovereign Soil
4,4.01,393147,10479.0,"The Crow (The Books of Pellinor, #3)"


**ADD BELOW TO FLASK TO PRIORITIZE RECOMMENDATIONS**

In [14]:
book_recs = pd.merge(book_recs, 
                     book_rating_aggs[['bookid','average_rating','ratings_count']],
                     left_on = 'iid',
                     right_on = 'bookid',
                     how = 'left'
                    )

In [15]:
import numpy as np
book_recs['log_rating_rank'] = book_recs['average_rating']*np.log(book_recs['ratings_count'])

In [16]:
book_recs.sort_values(by=['est', 
                          'ratings_count',
                          'average_rating'
                         ], ascending = [False, False, False])

,uid,iid,r_ui,est,details,title,bookid,average_rating,ratings_count,log_rating_rank
1,10015197,3,None,5.0,{'was_impossible': False},Harry Potter and the Sorcerer's Stone (Harry P...,3,4.46,5500888.0,69.221074
48,10015197,5907,None,5.0,{'was_impossible': False},The Hobbit or There and Back Again,5907,4.26,2329512.0,62.456581
1255,10015197,34,None,5.0,{'was_impossible': False},The Fellowship of the Ring (The Lord of the Ri...,34,4.35,1981328.0,63.071859
467,10015197,100915,None,5.0,{'was_impossible': False},"The Lion, the Witch and the Wardrobe (Chronicl...",100915,4.21,1783591.0,60.599326
1241,10015197,13496,None,5.0,{'was_impossible': False},"A Game of Thrones (A Song of Ice and Fire, #1)",13496,4.45,1555948.0,63.446300
939,10015197,1617,None,5.0,{'was_impossible': False},"Night (The Night Trilogy, #1)",1617,4.31,820759.0,58.693514
1434,10015197,2998,None,5.0,{'was_impossible': False},The Secret Garden,2998,4.13,715567.0,55.675830
190,10015197,7126,None,5.0,{'was_impossible': False},The Count of Monte Cristo,7126,4.23,612246.0,56.364282
1076,10015197,7244,None,5.0,{'was_impossible': False},The Poisonwood Bible,7244,4.04,585356.0,53.651101
955,10015197,234225,None,5.0,{'was_impossible': False},"Dune (Dune Chronicles, #1)",234225,4.21,547375.0,55.626264


In [17]:
# Book IDs to drop:
droplist = ['2657','4667024','77203','19063','1984','30119','18143977','18007564','32075671',
            '5470','8664353','10210','7445','19543','23772','19486412','8127','14935','332613',
            '18774964', '113946','11486','191139','77727','4406','21853621','7784','14891',
            '43070','99107','1420','168668'
           ]

In [18]:
book_recs[book_recs['bookid'].isin(droplist) == False]

,uid,iid,r_ui,est,details,title,bookid,average_rating,ratings_count,log_rating_rank
0,10015197,13722513,None,5.0,{'was_impossible': False},"Cinder (The Lunar Chronicles, #1)",13722513,4.15,8199.0,37.398835
1,10015197,3,None,5.0,{'was_impossible': False},Harry Potter and the Sorcerer's Stone (Harry P...,3,4.46,5500888.0,69.221074
2,10015197,96259,None,5.0,{'was_impossible': False},Crush,96259,4.30,14998.0,41.347390
3,10015197,23228,None,5.0,{'was_impossible': False},Boy Meets Boy,23228,3.85,56253.0,42.109816
4,10015197,12570845,None,5.0,{'was_impossible': False},"Blood Warrior (The Alexa Montgomery Saga, #1)",12570845,3.88,3834.0,32.016456
5,10015197,6408523,None,5.0,{'was_impossible': False},"Natsume's Book of Friends, Volume 1",6408523,4.23,5300.0,36.274205
6,10015197,37562415,None,5.0,{'was_impossible': False},Broken Circle (Gray Ghost #1),37562415,4.65,74.0,20.013903
7,10015197,278237,None,5.0,{'was_impossible': False},Parker Pyne Investigates,278237,3.76,11293.0,35.088088
8,10015197,7137327,None,5.0,{'was_impossible': False},"Enclave (Razorland, #1)",7137327,3.92,58482.0,43.027779
9,10015197,395220,None,5.0,{'was_impossible': False},Zami: A New Spelling of My Name,395220,4.24,10993.0,39.453259


In [19]:
book_recs = book_recs.sort_values(by=['iid','log_rating_rank'], ascending = False)
book_recs

,uid,iid,r_ui,est,details,title,bookid,average_rating,ratings_count,log_rating_rank
750,10015197,99702,None,5.0,{'was_impossible': False},"Island in the Sea of Time (Nantucket, #1)",99702,4.01,6297.0,35.078793
209,10015197,9969857,None,5.0,{'was_impossible': False},Doctor Who: Touched By An Angel,9969857,4.10,2089.0,31.342207
1095,10015197,99298,None,5.0,{'was_impossible': False},"The Harry Potter Collection 1-4 (Harry Potter,...",99298,4.67,46669.0,50.206401
696,10015197,991197,None,5.0,{'was_impossible': False},The Complete Persepolis,991197,4.38,86341.0,49.783342
266,10015197,99107,None,5.0,{'was_impossible': False},"Winnie-the-Pooh (Winnie-the-Pooh, #1)",NaN,NaN,NaN,NaN
893,10015197,98920,None,5.0,{'was_impossible': False},The Bastard of Istanbul,98920,3.81,17799.0,37.288080
1195,10015197,98865,None,5.0,{'was_impossible': False},Nan Goldin: The Ballad of Sexual Dependency,98865,4.21,3634.0,34.513956
1619,10015197,9876989,None,5.0,{'was_impossible': False},"Blue Exorcist, Vol. 1 (Blue Exorcist, #1)",9876989,4.37,45026.0,46.824530
599,10015197,9871439,None,5.0,{'was_impossible': False},The Word for World is Forest,9871439,3.95,19.0,11.630534
574,10015197,9860471,None,5.0,{'was_impossible': False},Angelmaker,9860471,3.94,379.0,23.393893


**END NOTEBOOK**

***